# Setup

In [ ]:
import os
os.environ["XLA_FLAGS"] = "--xla_force_host_platform_device_count=8"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import library

In [ ]:
from google.colab import files, output

!pip3 install vncorenlp
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!wget https://github.com/vncorenlp/VnCoreNLP/blob/master/models/postagger/vi-tagger
!wget -O vn_syllables.txt https://raw.githubusercontent.com/HienBM/sentiment_analysis_with_deep_learning/main/hieuthi-all-vietnamese-syllables.txt
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install emot
!pip install emoji
!pip install pyvi
!pip install transformers
!pip install torch
!pip install dataloader
!pip install fairseq
!pip install keras-tuner -q
!pip install tensorflow_addons
!pip install unidecode
!pip install py_vncorenlp
!pip install dash
!pip install google_play_scraper
!pip install pandas
!pip install wordcloud matplotlib
!pip install underthesea
output.clear()

# Import dictionary

In [ ]:
import pandas as pd
import requests
import re

def clean_char(text):
    char = str(text)
    char = re.sub('-', ' ', char)
    return char

def remove_vn_accent(old):
    new = str(old)
    new = new.lower()
    new = re.sub(r'[aáàảảãăắằẳẵặâấầẩẫậ]', 'a', new)
    new = re.sub(r'[eéèẻẽẹêếềểễệ]', 'e', new)
    new = re.sub(r'[iíìỉĩị]', 'i', new)
    new = re.sub(r'[oóòỏõọôốồổỗộơớờởỡợ]', 'o', new)
    new = re.sub(r'[uúùủũụưứừửữự]', 'u', new)
    new = re.sub(r'[yýỳỷỹỵ]', 'y', new)
    new = re.sub(r'[đ]', 'd', new)
    return new

def length_content(reviews):
    length_content = len(reviews)
    return length_content

def num_word(reviews):
    num_word = len(reviews.split())
    return num_word

# Fetch the text data from the URL
url = "https://raw.githubusercontent.com/HienBM/sentiment_analysis_with_deep_learning/main/Viet39K.txt"
response = requests.get(url)
text_data = response.text

# Split the text into lines
lines = text_data.split('\n')

# Create a DataFrame
vietnamese_word_dict = pd.DataFrame(lines, columns=['Word'])

viet_dict = pd.DataFrame(vietnamese_word_dict['Word'].apply(clean_char)).drop_duplicates()
viet_dict['no_acents'] = viet_dict['Word'].apply(remove_vn_accent)
viet_dict['check'] = (viet_dict['Word'] == viet_dict['no_acents'])
viet_dict['num_word'] = viet_dict['Word'].apply(num_word)
viet_dict['length_word'] = viet_dict['Word'].apply(length_content)
viet_dict = viet_dict.query('check != True and num_word > 1').sort_values(by='length_word', ascending=False)
viet_dict = viet_dict.drop_duplicates(subset=['no_acents'], keep=False)
adding_accents_dict = dict(zip(viet_dict['Word'], viet_dict['no_acents']))

# dictionary en2vi
en2vi = pd.read_csv('https://raw.githubusercontent.com/HienBM/text_normalization/main/top_500_adjective_eng.txt', delimiter='\t')
en2vi_dict = dict(zip(en2vi['Adj'], en2vi['Mean']))

# dictionary teencode
teencode = pd.read_csv('https://raw.githubusercontent.com/HienBM/sentiment_analysis_with_deep_learning/main/teencode_misspelling.txt', delimiter='\t', header=None)
teencode_dict = dict(zip(teencode.iloc[:,0], teencode.iloc[:,1]))

# dictionary vietnamese offensive
vn_offensive = pd.read_csv('https://raw.githubusercontent.com/HienBM/sentiment_analysis_with_deep_learning/main/vn_offensive.txt', delimiter='\t', header=None)
vn_offensive_dict = dict(zip(vn_offensive.iloc[:,0], vn_offensive.iloc[:,1]))


# Prepare dataset

In [ ]:
import pandas as pd
df = pd.read_csv('Data/train.csv')
df = df.drop_duplicates()

In [ ]:
df.shape

(5172, 8)

In [ ]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5172 entries, 0 to 5171
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_name         5172 non-null   object 
 1   post_message      5171 non-null   object 
 2   timestamp_post    5008 non-null   object 
 3   num_like_post     5026 non-null   object 
 4   num_comment_post  5158 non-null   object 
 5   num_share_post    4243 non-null   object 
 6   label             5172 non-null   int64  
 7   id                4372 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 363.7+ KB
None


,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,id
0,2167074723833126912,"Cần các bậc phụ_huynh xã Ngũ_Thái lên_tiếng , ...",1584426000.0,45,15,8,1,NaN
1,7368497547812410368,KÊU_GỌI ĂN_CHAY CẦU_NGUYỆN XIN CHÚA CỨU KHỎI D...,1581362736.0,979,39,138,1,NaN
2,-5754142625280308224,"Giàn khoan dầu_khí gặp sự_cố , chết người !Già...",1587041580.0,85,13,61,1,NaN
3,4177935308849652224,"Thuận_Lợi có ca dương_tính CV19 rồi đó , mọi n...",1596449280.0,114,12,5,1,NaN
4,5540309800745996288,Sa_Pa cho 9 người khách nước_ngoài đi cùng chu...,1583548080.0,166,4,21,1,NaN


In [ ]:
pd.DataFrame(df['label'].value_counts())

,label
0,4238
1,934


In [ ]:
df.isnull().sum()

user_name             0
post_message          1
timestamp_post      164
num_like_post       146
num_comment_post     14
num_share_post      929
label                 0
id                  800
dtype: int64

In [ ]:
filtered_df = df.loc[df['post_message'].isnull()]
# In kết quả
print(filtered_df)

                             user_name post_message timestamp_post  \
3047  66a74969be46766c182b99dc01740a11          NaN     1585331222   

     num_like_post num_comment_post num_share_post  label      id  
3047          9717               28           88.0      0  2248.0  


In [ ]:
df = df.dropna(subset=['post_message'])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5171 entries, 0 to 5171
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_name         5171 non-null   object 
 1   post_message      5171 non-null   object 
 2   timestamp_post    5007 non-null   object 
 3   num_like_post     5025 non-null   object 
 4   num_comment_post  5157 non-null   object 
 5   num_share_post    4242 non-null   object 
 6   label             5171 non-null   int64  
 7   id                4371 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 363.6+ KB
None


In [ ]:
# Data k có giá trị null
df_dropna = df.dropna()
print(f"Số dòng trước khi loại bỏ: {len(df)}")
print(f"Số dòng sau khi loại bỏ: {len(df_dropna)}")

Số dòng trước khi loại bỏ: 5171
Số dòng sau khi loại bỏ: 3521


In [ ]:
df.isnull().sum()

user_name             0
post_message          1
timestamp_post      164
num_like_post       146
num_comment_post     14
num_share_post      929
label                 0
id                  800
dtype: int64

In [ ]:
column_name = 'num_like_post'
unknown_count = df[df[column_name] == 'unknown'].shape[0]

print(f"Số dòng có giá trị 'unknown' trong cột '{column_name}': {unknown_count}")

Số dòng có giá trị 'unknown' trong cột 'num_like_post': 121


In [ ]:
column_name = 'num_comment_post'
unknown_count = df[df[column_name] == 'unknown'].shape[0]

print(f"Số dòng có giá trị 'unknown' trong cột '{column_name}': {unknown_count}")

Số dòng có giá trị 'unknown' trong cột 'num_comment_post': 121


In [ ]:
column_name = 'num_share_post'
unknown_count = df[df[column_name] == 'unknown'].shape[0]

print(f"Số dòng có giá trị 'unknown' trong cột '{column_name}': {unknown_count}")

Số dòng có giá trị 'unknown' trong cột 'num_share_post': 121


In [ ]:
df[df['num_share_post'] == 'unknown']
# => có 121 dòng có chứa các giá trị num_like_post, num_comment_post. num_share_post là unknown

,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,id
5,7649993800071920640,Mọi người ra đường nhớ đeo khẩu_trang nha .Đã ...,1596949200.0,unknown,unknown,unknown,1,NaN
11,-8578563326025052160,Danh_Hài Thuý_Nga_Qua_Đời Để Lại_Gia_Tài_Triệu...,1593485100.0,unknown,unknown,unknown,1,NaN
12,107031663370279008,Quanh khu_vực Xuân_Mai mọi người cẩn_trọng chú...,1589980200.0,unknown,unknown,unknown,1,NaN
14,-9222677872199538688,BAN MUỐN MÌNH LÀ NGƯỜI SỐNG_SÓT TRONG CON_SỐ 9...,1582534740.0,unknown,unknown,unknown,1,NaN
19,-4754275936642500608,Vụ bắt_cóc con_nít thôn 5 nông Sơn sao k thấy ...,1586996520.0,unknown,unknown,unknown,1,NaN
...,...,...,...,...,...,...,...,...
5015,00e31592dda2b69862316c49d71e2c01,Mọi người ơi hạn_chế za đường nhé vì 3nguoi nh...,1580958840.0,unknown,unknown,unknown,1,4216.0
5049,64eb8fe84f9b485967d7334f934a6348,Thông_báo Bệnh_viện Thái_Hoà Hồng_Ngự mới tiếp...,1583741700.0,unknown,unknown,unknown,1,4250.0
5123,30ba11e2af4156a2a4d6870cdecda3da,"TIN NÓNG , ĐÃ TÌM RA NGUYÊN NHIÊN GÂY TỬ_VONG ...",1566090000.0,unknown,unknown,unknown,1,4324.0
5149,b6041ccf83b87019f42bf93281b3b110,Xong ...!Ning_Giang_Toang rồi Ninh_Giang dính ...,1595778900.0,unknown,unknown,unknown,1,4350.0


In [ ]:
df_dropna [df_dropna ['num_share_post'] == 'unknown']

,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,id
835,fe8176d5bc081419f9b08e195525c131,Người mặt đen miệng ngậm 2 quả gì đó tay cầm đ...,1575711720.0,unknown,unknown,unknown,1,36.0
877,45d80e8d62eac8e3f28a8c822fab002d,Mọi người ra đường nhớ đeo khẩu_trang nha .Đã ...,1596949200.0,unknown,unknown,unknown,1,78.0
977,0adfdeb881147078b0c50354193d67dd,Ca_Phẫu_Thuật thất_bại .Hoài_Linh lâm vào tình...,1501295400.0,unknown,unknown,unknown,1,178.0
1085,6d46f2b04842bb2221e493968cd05ca4,Ngày hôm_qua gia_đình mình có ra chợ Thới_Bình...,1499075340.0,unknown,unknown,unknown,1,286.0
1437,17fdafcfe5348b4c35f01f62c403e68c,Danh_Hài Thuý_Nga_Qua_Đời Để Lại_Gia_Tài_Triệu...,1593485100.0,unknown,unknown,unknown,1,638.0
...,...,...,...,...,...,...,...,...
5015,00e31592dda2b69862316c49d71e2c01,Mọi người ơi hạn_chế za đường nhé vì 3nguoi nh...,1580958840.0,unknown,unknown,unknown,1,4216.0
5049,64eb8fe84f9b485967d7334f934a6348,Thông_báo Bệnh_viện Thái_Hoà Hồng_Ngự mới tiếp...,1583741700.0,unknown,unknown,unknown,1,4250.0
5123,30ba11e2af4156a2a4d6870cdecda3da,"TIN NÓNG , ĐÃ TÌM RA NGUYÊN NHIÊN GÂY TỬ_VONG ...",1566090000.0,unknown,unknown,unknown,1,4324.0
5149,b6041ccf83b87019f42bf93281b3b110,Xong ...!Ning_Giang_Toang rồi Ninh_Giang dính ...,1595778900.0,unknown,unknown,unknown,1,4350.0


In [ ]:
df[df['num_share_post'] == 'unknown']['label'].value_counts()

1    118
0      3
Name: label, dtype: int64

# Add num_word and num_char column

In [ ]:
df['num_word'] = df['post_message'].apply(str).apply(num_word)
df['num_char'] = df['post_message'].apply(str).apply(length_content)

In [ ]:
df

,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,id,num_word,num_char
0,2167074723833126912,"Cần các bậc phụ_huynh xã Ngũ_Thái lên_tiếng , ...",1584426000.0,45,15,8,1,NaN,34,182
1,7368497547812410368,KÊU_GỌI ĂN_CHAY CẦU_NGUYỆN XIN CHÚA CỨU KHỎI D...,1581362736.0,979,39,138,1,NaN,10,62
2,-5754142625280308224,"Giàn khoan dầu_khí gặp sự_cố , chết người !Già...",1587041580.0,85,13,61,1,NaN,41,212
3,4177935308849652224,"Thuận_Lợi có ca dương_tính CV19 rồi đó , mọi n...",1596449280.0,114,12,5,1,NaN,28,151
4,5540309800745996288,Sa_Pa cho 9 người khách nước_ngoài đi cùng chu...,1583548080.0,166,4,21,1,NaN,13,65
...,...,...,...,...,...,...,...,...,...,...
5167,20933f35ef5d22b4d8193cc269c8ff1e,BÀ MẸ VIỆT_NAM ANH_HÙNG 95 TUỔI MAY KHẨU_TRANG...,1584795126.0,5800,1300,12000,0,4368.0,260,1366
5168,a117312f796a22e364b8e241b8cb91eb,Nguồn cung khan kiếm nhưng nhu_cầu cao tạo áp_...,1590645643,21,1,NaN,0,4369.0,26,139
5169,547ba1b4f95ec07f2cdada24a6eec693,Lời cảnh_tỉnh cho các thanh_niên dân TỔ ... tố...,1589774421.0,3,1,NaN,1,4370.0,38,177
5170,acb4a36d6247a0c89dac880725b2b3a0,Đến bây_giờ mới biết chỉ cần học lái_xe hạng B...,1589551407.0,144,38,87,1,4371.0,30,142


# Add num_hashtag và num_url column

In [ ]:
def count_hashtags(text):
    count = text.count('#')
    return count

def count_url(text):
    count1 = text.count('url')
    count2 = text.count('URL')
    count = count1 + count2
    return count

In [ ]:
df['num_hashtag'] = df['post_message'].apply(str).apply(count_hashtags)
df['num_url'] = df['post_message'].apply(str).apply(count_url)

In [ ]:
df.head()

,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,id,num_word,num_char,num_hashtag,num_url
0,2167074723833126912,"Cần các bậc phụ_huynh xã Ngũ_Thái lên_tiếng , ...",1584426000.0,45,15,8,1,NaN,34,182,0,0
1,7368497547812410368,KÊU_GỌI ĂN_CHAY CẦU_NGUYỆN XIN CHÚA CỨU KHỎI D...,1581362736.0,979,39,138,1,NaN,10,62,0,0
2,-5754142625280308224,"Giàn khoan dầu_khí gặp sự_cố , chết người !Già...",1587041580.0,85,13,61,1,NaN,41,212,0,0
3,4177935308849652224,"Thuận_Lợi có ca dương_tính CV19 rồi đó , mọi n...",1596449280.0,114,12,5,1,NaN,28,151,0,0
4,5540309800745996288,Sa_Pa cho 9 người khách nước_ngoài đi cùng chu...,1583548080.0,166,4,21,1,NaN,13,65,0,0


In [ ]:
pd.DataFrame(df['num_hashtag'].value_counts())

,num_hashtag
0,4582
1,143
2,109
3,95
4,78
5,45
6,31
7,22
8,10
9,9


In [ ]:
pd.DataFrame(df['num_url'].value_counts())

,num_url
0,4177
1,548
2,301
3,64
4,30
5,21
6,9
9,4
7,4
8,4


# Preprocessing text

In [ ]:
import pandas as pd
from unicodedata import digit
import re
import string
import emot
from google.colab import output
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
import nltk
nltk.download('punkt')
from tqdm import tqdm
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pickle
import os
import warnings
warnings.filterwarnings("ignore")
import py_vncorenlp
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
from collections import Counter

# Preprocessing
def remove_duplicate_emoji(orig_str):
    prev_emoji = None
    remove_duplicate_emoji = []
    for c in orig_str:
        if c in UNICODE_EMOJI:
            if prev_emoji == c:
                continue
            prev_emoji = c
        remove_duplicate_emoji.append(c)
    return "".join(remove_duplicate_emoji)

def preprocessing_vietnamese(text):

    vietnamese_vovel = list('aáàảãạăắằẳẵặâấầẩẫậeéèẻẽẹêếềểễệiíìỉĩịoóòỏõọôốồổỗộơớờởỡợuúùủũụưứừửữựyýỳỷỹỵđ')
    ascii_lowercase = list(string.ascii_lowercase)
    ascii_uppercase = list(string.ascii_uppercase)
    digits = list(string.digits)
    punctuation = list('!;?,.')
    whitespace = list(' ')
    digits_dict = {'0':'không','1':'một','2':'hai','3':'ba','4':'bốn','5':'năm','6':'sáu','7':'bảy','8':'tám','9':'chín'}
    double_consonant = ['bb', 'cc', 'dd', 'ff', 'gg', 'kk', 'll', 'mm', 'nn', 'pp', 'rr', 'ss', 'tt', 'uu', 'vv', 'yy', 'zz']
    double_vowel = ['aa', 'ee', 'ii', 'oo', 'uu']
    punctuation_1 = list('"#$%&\'()*+-/:<=>@[\\]^_`{|}~')

    emoticon = [x for x in EMOTICONS_EMO]
    emoji = [x for x in UNICODE_EMOJI]

    # remove char
    accept_char = vietnamese_vovel + ascii_lowercase + emoticon + emoji + digits

    # remove hastag, mention, url
    text = re.sub(r'<[^<]+?>','',text)
    text = re.sub("@[A-Za-z0-9_]+","", text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    text = re.sub('http[s]?://\S+', '', text)

    # convert text to lowercase
    text = text.lower()

    # đổi các từ liên quan thành covid
    text = text.replace('covid-19', 'covid')
    text = text.replace('ncov', 'covid')
    text = text.replace('cv19', 'covid')
    text = text.replace('corona', 'covid')
    text = text.replace("convid", 'covid')


    # remove special characters
    # text = [letter if letter not in accept_char else ' ' for letter in text]
    text = [letter if letter in accept_char else ' ' for letter in text]
    text = ''.join(text)

    # normalize elongated word
    text = re.sub(r'([a-z0-9]+?)\1\1+', r'\1', text)

    # remove duplicate and continuous words/phrases
    text = re.sub(r'\b(.+)(\s+\1\1\b)+', r'\1\1', text)

    # split emoji
    for k,v in emot.UNICODE_EMOJI.items():
        text = text.replace(k, ' '+k+' ')

    # remove duplicate emoji
    text = remove_duplicate_emoji(text)

    # create whitespace in start and end of sentence
    text_len = len(text) + 4
    text = text.center(text_len)

    # mapping teencode
    for k,v in teencode_dict.items():
        key = ' '+k+' '
        value = ' '+v+' '
        text = text.replace(str(key), str(value))

    # mapping vn_offensive
    for k, v in vn_offensive_dict.items():
        key = ' '+k+' '
        value = ' '+v+' '
        text = text.replace(key, value)

    # adding vietnamese accents
    for k, v in adding_accents_dict.items():
        key = ' '+k+' '
        value = ' '+v+' '
        text = text.replace(value, key)

    # remove extra whitespace
    text = re.sub(r'^\s+$|\s+$', ' ', text).strip()

    # remove tab
    text = re.sub(r'^\s*|\s\s*', ' ', text).strip()

    # tokenize vnlp
    sentences = rdrsegmenter.tokenize(text)
    for sentence in sentences:
        return " ".join(sentence)

    # tokenize
    # text = ViTokenizer.tokenize(text)
    return text

output.clear()

In [ ]:
def normalizePost(post, tweet_tokenizer, vncorenlp, is_preprocessing = True,
                  use_segment=False, remove_punc_stopword=False, lowercase_opt=False,
                  truncation_method="head_only", length=256):
    if is_preprocessing:
      post = post.strip()
      URL_pattern = r"(?:http?s://|www.)[^\"]+"
      hashtag_pattern = r"#\w+"

      post = re.sub(URL_pattern, "", post)
      post = re.sub(hashtag_pattern, "hashtag", post)
      post = re.sub("\.+",".", post)
      # post = re.sub("#\s+", " ", post)
      post = re.sub("\*+", " ", post)
      post = re.sub("\$+", "đô ", post)
      post = re.sub("-{2,}", "", post)
      post = re.sub("\@+", "", post)
      post = re.sub("\[[0-9]?[0-9]]", " : dẫn_chứng ", post)

    post = post.strip()
    if lowercase_opt:
      post = post.lower()
    tokens = tweet_tokenizer.tokenize(post.replace("’", "'").replace("…", "..."))

    post = " ".join(tokens)
    if use_segment:
        tokens = vncorenlp.tokenize(post.replace("’", "'").replace("…", "..."))
        tokens = [t for ts in tokens for t in ts]
    normPost = " ".join(tokens)

    if remove_punc_stopword:
      tokens = [t for t in normPost if not t in vnmese_stopwords]
    normPost = " ".join(tokens)

    normPost = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normPost)
    normPost = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normPost)
    normPost = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normPost)

    if use_segment:
        normPost = normPost.replace('< url >', '<url>')
        normPost = re.sub(r"# (\w+)", r'#\1', normPost)
    if truncation_method == "head_only":
      normPost = " ".join(normPost.split(" ")[:length])
    if truncation_method == "tail_only":
      normPost = " ".join(normPost.split(" ")[-length:])
    if truncation_method == "head_tail":
      normPost = " ".join(normPost.split(" ")[:int(length*0.25)]) + " " +  " ".join(normPost.split(" ")[-int(length*0.75):])

    replace_list = json.load(open("/content/drive/MyDrive/replace_list.txt"))
    for k, v in replace_list.items():
        normPost = normPost.replace(k, v)
    return normPost

In [ ]:
preprocessing_vietnamese('covid-19 việt nam và xã vung tau sa pa có hang hoa da_dang, @ --- chat luong tuyet voi, nhung ma giao_hang cham qua, nhu cai dau buoi k =)) 🤬🤬🤬🤬')

'covid việt_nam và xã vũng_tàu sa_pa có hàng_hoá đa_dạng chất_lượng tuyệt_vời nhưng_mà giao hàng cham qua nhu cai đầu buồi không 🤬'

In [ ]:
df['preprocess'] = df['post_message'].apply(str).apply(preprocessing_vietnamese)
df = df.query('''preprocess != "" ''')

In [ ]:
df.to_csv('Data/data_after_text_preprocessing.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5091 entries, 0 to 5171
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_name         5091 non-null   object 
 1   post_message      5090 non-null   object 
 2   timestamp_post    4927 non-null   object 
 3   num_like_post     4953 non-null   object 
 4   num_comment_post  5077 non-null   object 
 5   num_share_post    4214 non-null   object 
 6   label             5091 non-null   int64  
 7   id                4310 non-null   float64
 8   num_word          5091 non-null   int64  
 9   num_char          5091 non-null   int64  
 10  num_hashtag       5091 non-null   int64  
 11  num_url           5091 non-null   int64  
 12  preprocess        5091 non-null   object 
dtypes: float64(1), int64(5), object(7)
memory usage: 685.9+ KB


# Loại các dòng có Tensor size lớn hơn [1, 256] cho SA



In [ ]:
from transformers import RobertaForSequenceClassification

model_name = "wonrax/phobert-base-vietnamese-sentiment"
model = RobertaForSequenceClassification.from_pretrained(model_name)
max_position_embeddings = model.config.max_position_embeddings

print("Kích thước tối đa của mô hình:", max_position_embeddings)

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Kích thước tối đa của mô hình: 258


In [ ]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Hàm kiểm tra kích thước Tensor
def check_tensor_size(sentence):
    model_name = "wonrax/phobert-base-vietnamese-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    input_ids = torch.tensor([tokenizer.encode(sentence)])
    tensor_size = input_ids.size()
    return tensor_size

# Loại bỏ các dòng có kích thước Tensor lớn hơn [1, 258]
sa_data = df.drop(df[df['preprocess'].apply(check_tensor_size).apply(lambda x: x > torch.Size([1, 256]))].index)

In [ ]:
sa_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4271 entries, 0 to 5171
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_name         4271 non-null   object 
 1   post_message      4270 non-null   object 
 2   timestamp_post    4115 non-null   object 
 3   num_like_post     4161 non-null   object 
 4   num_comment_post  4259 non-null   object 
 5   num_share_post    3504 non-null   object 
 6   label             4271 non-null   int64  
 7   id                3629 non-null   float64
 8   num_word          4271 non-null   int64  
 9   num_char          4271 non-null   int64  
 10  num_hashtag       4271 non-null   int64  
 11  num_url           4271 non-null   int64  
 12  preprocess        4271 non-null   object 
dtypes: float64(1), int64(5), object(7)
memory usage: 467.1+ KB


In [ ]:
sa_data.head()

,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,id,num_word,num_char,num_hashtag,num_url,preprocess
0,2167074723833126912,"Cần các bậc phụ_huynh xã Ngũ_Thái lên_tiếng , ...",1584426000.0,45,15,8,1,NaN,34,182,0,0,cần các bậc phụ_huynh xã ngũ thái lên_tiếng kh...
1,7368497547812410368,KÊU_GỌI ĂN_CHAY CẦU_NGUYỆN XIN CHÚA CỨU KHỎI D...,1581362736.0,979,39,138,1,NaN,10,62,0,0,kêu_gọi ăn_chay cầu_nguyện xin chúa cứu khỏi d...
2,-5754142625280308224,"Giàn khoan dầu_khí gặp sự_cố , chết người !Già...",1587041580.0,85,13,61,1,NaN,41,212,0,0,giàn khoan dầu_khí gặp sự_cố chết người giàn k...
3,4177935308849652224,"Thuận_Lợi có ca dương_tính CV19 rồi đó , mọi n...",1596449280.0,114,12,5,1,NaN,28,151,0,0,thuận_lợi có ca dương_tính cv19 rồi đó mọi ngư...
4,5540309800745996288,Sa_Pa cho 9 người khách nước_ngoài đi cùng chu...,1583548080.0,166,4,21,1,NaN,13,65,0,0,sa_pa cho 9 người khách nước_ngoài đi cùng chu...


# SA

In [ ]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification

def predict_sentiment(sentence):
    model_name = "wonrax/phobert-base-vietnamese-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    model = RobertaForSequenceClassification.from_pretrained(model_name)

    input_ids = torch.tensor([tokenizer.encode(sentence)])

    with torch.no_grad():
        out = model(input_ids)
        probabilities = out.logits.softmax(dim=-1).tolist()[0]
    return probabilities

In [ ]:
# test hàm predict_sentiment
import pandas as pd

# Tạo dataframe mẫu
data = {'post_message': ['Đây là một câu tích cực.', 'cửa_hàng giao_hàng rất nhanh', 'Có gì đó ở giữa.', 'Câu tiếp theo tích cực nhỉ?']}
df_sa_test = pd.DataFrame(data)

# Dự đoán cảm xúc cho cột "post_message"
df_sa_test[['NEG', 'POS', 'NEU']] = df_sa_test['post_message'].apply(predict_sentiment).apply(pd.Series)

# In kết quả
print(df_sa_test)

                   post_message       NEG       POS       NEU
0      Đây là một câu tích cực.  0.037491  0.560729  0.401780
1  cửa_hàng giao_hàng rất nhanh  0.001731  0.981409  0.016860
2              Có gì đó ở giữa.  0.053741  0.078375  0.867884
3   Câu tiếp theo tích cực nhỉ?  0.002722  0.988118  0.009160


In [ ]:
sa_data[['NEG', 'POS', 'NEU']] = sa_data['preprocess'].apply(predict_sentiment).apply(pd.Series)
sa_data

,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,id,num_word,num_char,num_hashtag,num_url,preprocess,NEG,POS,NEU
0,2167074723833126912,"Cần các bậc phụ_huynh xã Ngũ_Thái lên_tiếng , ...",1584426000.0,45,15,8,1,NaN,34,182,0,0,cần các bậc phụ_huynh xã ngũ thái lên_tiếng kh...,0.985327,0.004864,0.009808
1,7368497547812410368,KÊU_GỌI ĂN_CHAY CẦU_NGUYỆN XIN CHÚA CỨU KHỎI D...,1581362736.0,979,39,138,1,NaN,10,62,0,0,kêu_gọi ăn_chay cầu_nguyện xin chúa cứu khỏi d...,0.659123,0.213645,0.127232
2,-5754142625280308224,"Giàn khoan dầu_khí gặp sự_cố , chết người !Già...",1587041580.0,85,13,61,1,NaN,41,212,0,0,giàn khoan dầu_khí gặp sự_cố chết người giàn k...,0.929886,0.016793,0.053320
3,4177935308849652224,"Thuận_Lợi có ca dương_tính CV19 rồi đó , mọi n...",1596449280.0,114,12,5,1,NaN,28,151,0,0,thuận_lợi có ca dương_tính cv19 rồi đó mọi ngư...,0.004563,0.982044,0.013393
4,5540309800745996288,Sa_Pa cho 9 người khách nước_ngoài đi cùng chu...,1583548080.0,166,4,21,1,NaN,13,65,0,0,sa_pa cho 9 người khách nước_ngoài đi cùng chu...,0.038136,0.090191,0.871673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5166,1fa13b99e3b37d346a2b8cc7b89260ce,"Một vị tướng , một đô_đốc lừng_lẫy ... nay đứn...",1589942616.0,58,25,21,0,4367.0,41,238,0,0,một vị tướng một đô_đốc lừng_lẫy nay đứng trướ...,0.959250,0.008172,0.032578
5168,a117312f796a22e364b8e241b8cb91eb,Nguồn cung khan kiếm nhưng nhu_cầu cao tạo áp_...,1590645643,21,1,NaN,0,4369.0,26,139,0,0,nguồn cung khan kiếm nhưng nhu_cầu cao tạo áp_...,0.003123,0.974172,0.022705
5169,547ba1b4f95ec07f2cdada24a6eec693,Lời cảnh_tỉnh cho các thanh_niên dân TỔ ... tố...,1589774421.0,3,1,NaN,1,4370.0,38,177,0,1,lời cảnh_tỉnh cho các thanh_niên dân tổ tốc_độ...,0.975487,0.006836,0.017676
5170,acb4a36d6247a0c89dac880725b2b3a0,Đến bây_giờ mới biết chỉ cần học lái_xe hạng B...,1589551407.0,144,38,87,1,4371.0,30,142,0,0,đến bây_giờ mới biết chỉ cần học lái_xe hạng b...,0.085258,0.719443,0.195299


In [ ]:
sa_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4271 entries, 0 to 5171
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_name         4271 non-null   object 
 1   post_message      4270 non-null   object 
 2   timestamp_post    4115 non-null   object 
 3   num_like_post     4161 non-null   object 
 4   num_comment_post  4259 non-null   object 
 5   num_share_post    3504 non-null   object 
 6   label             4271 non-null   int64  
 7   id                3629 non-null   float64
 8   num_word          4271 non-null   int64  
 9   num_char          4271 non-null   int64  
 10  num_hashtag       4271 non-null   int64  
 11  num_url           4271 non-null   int64  
 12  preprocess        4271 non-null   object 
 13  NEG               4271 non-null   float64
 14  POS               4271 non-null   float64
 15  NEU               4271 non-null   float64
dtypes: float64(4), int64(5), object(7)
memory 

In [ ]:
filtered_df = sa_data.loc[sa_data['post_message'].isnull()]
# In kết quả
print(filtered_df)

                             user_name post_message timestamp_post  \
3047  66a74969be46766c182b99dc01740a11          NaN     1585331222   

     num_like_post num_comment_post num_share_post  label      id  num_word  \
3047          9717               28           88.0      0  2248.0         1   

      num_char  num_hashtag  num_url preprocess       NEG       POS       NEU  
3047         3            0        0        nan  0.976525  0.005358  0.018117  


In [ ]:
sa_data = sa_data.dropna(subset=['post_message'])
print(sa_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4270 entries, 0 to 5171
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_name         4270 non-null   object 
 1   post_message      4270 non-null   object 
 2   timestamp_post    4114 non-null   object 
 3   num_like_post     4160 non-null   object 
 4   num_comment_post  4258 non-null   object 
 5   num_share_post    3503 non-null   object 
 6   label             4270 non-null   int64  
 7   id                3628 non-null   float64
 8   num_word          4270 non-null   int64  
 9   num_char          4270 non-null   int64  
 10  num_hashtag       4270 non-null   int64  
 11  num_url           4270 non-null   int64  
 12  preprocess        4270 non-null   object 
 13  NEG               4270 non-null   float64
 14  POS               4270 non-null   float64
 15  NEU               4270 non-null   float64
dtypes: float64(4), int64(5), object(7)
memory 

In [ ]:
sa_data.to_csv('Data/sa_data.csv')

# Handle timestamp

In [ ]:
# Data không có timestamp null và timestamp k phải số
df_timestamp = sa_data.dropna(subset=['timestamp_post'])
df_timestamp = df_timestamp[~df_timestamp['timestamp_post'].str.contains(r'[a-zA-Z]')]
print(df_timestamp.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4106 entries, 0 to 5171
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_name         4106 non-null   object 
 1   post_message      4106 non-null   object 
 2   timestamp_post    4106 non-null   object 
 3   num_like_post     3998 non-null   object 
 4   num_comment_post  4094 non-null   object 
 5   num_share_post    3339 non-null   object 
 6   label             4106 non-null   int64  
 7   id                3530 non-null   float64
 8   num_word          4106 non-null   int64  
 9   num_char          4106 non-null   int64  
 10  num_hashtag       4106 non-null   int64  
 11  num_url           4106 non-null   int64  
 12  preprocess        4106 non-null   object 
 13  NEG               4106 non-null   float64
 14  POS               4106 non-null   float64
 15  NEU               4106 non-null   float64
dtypes: float64(4), int64(5), object(7)
memory 

In [ ]:
from datetime import datetime

def transform_timestamp(timestamp):
    # Chuyển đổi chuỗi thành số thực
    timestamp_float = float(timestamp)

    # Lấy phần nguyên của số thực
    timestamp_int = int(timestamp_float)

    # Chuyển đổi timestamp thành đối tượng datetime
    dt = datetime.fromtimestamp(timestamp_int)

    # Lấy giá trị hour (giờ)
    hour = dt.hour

    # Lấy giá trị weekday (thứ trong tuần)
    weekday = dt.weekday()

    # Lấy giá trị day (ngày)
    day = dt.day

    # Lấy giá trị month (tháng)
    month = dt.month
    year = dt.year

    # Tạo list chứa các giá trị đã lấy
    result = [hour, weekday, day, month, year]

    # Trả về kết quả
    return result

In [ ]:
# test trên tập đã xử lý timestamp
df_timestamp[['hour', 'weekday', 'day', 'month', 'year']] = df_timestamp['timestamp_post'].astype(str).apply(transform_timestamp).apply(pd.Series)

,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,id,num_word,num_char,...,num_url,preprocess,NEG,POS,NEU,hour,weekday,day,month,year
0,2167074723833126912,"Cần các bậc phụ_huynh xã Ngũ_Thái lên_tiếng , ...",1584426000.0,45,15,8,1,NaN,34,182,...,0,cần các bậc phụ_huynh xã ngũ thái lên_tiếng kh...,0.985327,0.004864,0.009808,6,1,17,3,2020
1,7368497547812410368,KÊU_GỌI ĂN_CHAY CẦU_NGUYỆN XIN CHÚA CỨU KHỎI D...,1581362736.0,979,39,138,1,NaN,10,62,...,0,kêu_gọi ăn_chay cầu_nguyện xin chúa cứu khỏi d...,0.659123,0.213645,0.127232,19,0,10,2,2020
2,-5754142625280308224,"Giàn khoan dầu_khí gặp sự_cố , chết người !Già...",1587041580.0,85,13,61,1,NaN,41,212,...,0,giàn khoan dầu_khí gặp sự_cố chết người giàn k...,0.929886,0.016793,0.053320,12,3,16,4,2020
3,4177935308849652224,"Thuận_Lợi có ca dương_tính CV19 rồi đó , mọi n...",1596449280.0,114,12,5,1,NaN,28,151,...,0,thuận_lợi có ca dương_tính cv19 rồi đó mọi ngư...,0.004563,0.982044,0.013393,10,0,3,8,2020
4,5540309800745996288,Sa_Pa cho 9 người khách nước_ngoài đi cùng chu...,1583548080.0,166,4,21,1,NaN,13,65,...,0,sa_pa cho 9 người khách nước_ngoài đi cùng chu...,0.038136,0.090191,0.871673,2,5,7,3,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5166,1fa13b99e3b37d346a2b8cc7b89260ce,"Một vị tướng , một đô_đốc lừng_lẫy ... nay đứn...",1589942616.0,58,25,21,0,4367.0,41,238,...,0,một vị tướng một đô_đốc lừng_lẫy nay đứng trướ...,0.959250,0.008172,0.032578,2,2,20,5,2020
5168,a117312f796a22e364b8e241b8cb91eb,Nguồn cung khan kiếm nhưng nhu_cầu cao tạo áp_...,1590645643,21,1,NaN,0,4369.0,26,139,...,0,nguồn cung khan kiếm nhưng nhu_cầu cao tạo áp_...,0.003123,0.974172,0.022705,6,3,28,5,2020
5169,547ba1b4f95ec07f2cdada24a6eec693,Lời cảnh_tỉnh cho các thanh_niên dân TỔ ... tố...,1589774421.0,3,1,NaN,1,4370.0,38,177,...,1,lời cảnh_tỉnh cho các thanh_niên dân tổ tốc_độ...,0.975487,0.006836,0.017676,4,0,18,5,2020
5170,acb4a36d6247a0c89dac880725b2b3a0,Đến bây_giờ mới biết chỉ cần học lái_xe hạng B...,1589551407.0,144,38,87,1,4371.0,30,142,...,0,đến bây_giờ mới biết chỉ cần học lái_xe hạng b...,0.085258,0.719443,0.195299,14,4,15,5,2020


In [ ]:
df_timestamp.head()

,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,id,num_word,num_char,...,num_url,preprocess,NEG,POS,NEU,hour,weekday,day,month,year
0,2167074723833126912,"Cần các bậc phụ_huynh xã Ngũ_Thái lên_tiếng , ...",1584426000.0,45,15,8,1,NaN,34,182,...,0,cần các bậc phụ_huynh xã ngũ thái lên_tiếng kh...,0.985327,0.004864,0.009808,6,1,17,3,2020
1,7368497547812410368,KÊU_GỌI ĂN_CHAY CẦU_NGUYỆN XIN CHÚA CỨU KHỎI D...,1581362736.0,979,39,138,1,NaN,10,62,...,0,kêu_gọi ăn_chay cầu_nguyện xin chúa cứu khỏi d...,0.659123,0.213645,0.127232,19,0,10,2,2020
2,-5754142625280308224,"Giàn khoan dầu_khí gặp sự_cố , chết người !Già...",1587041580.0,85,13,61,1,NaN,41,212,...,0,giàn khoan dầu_khí gặp sự_cố chết người giàn k...,0.929886,0.016793,0.053320,12,3,16,4,2020
3,4177935308849652224,"Thuận_Lợi có ca dương_tính CV19 rồi đó , mọi n...",1596449280.0,114,12,5,1,NaN,28,151,...,0,thuận_lợi có ca dương_tính cv19 rồi đó mọi ngư...,0.004563,0.982044,0.013393,10,0,3,8,2020
4,5540309800745996288,Sa_Pa cho 9 người khách nước_ngoài đi cùng chu...,1583548080.0,166,4,21,1,NaN,13,65,...,0,sa_pa cho 9 người khách nước_ngoài đi cùng chu...,0.038136,0.090191,0.871673,2,5,7,3,2020


In [ ]:
df_timestamp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4106 entries, 0 to 5171
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_name         4106 non-null   object 
 1   post_message      4106 non-null   object 
 2   timestamp_post    4106 non-null   object 
 3   num_like_post     3998 non-null   object 
 4   num_comment_post  4094 non-null   object 
 5   num_share_post    3339 non-null   object 
 6   label             4106 non-null   int64  
 7   id                3530 non-null   float64
 8   num_word          4106 non-null   int64  
 9   num_char          4106 non-null   int64  
 10  num_hashtag       4106 non-null   int64  
 11  num_url           4106 non-null   int64  
 12  preprocess        4106 non-null   object 
 13  NEG               4106 non-null   float64
 14  POS               4106 non-null   float64
 15  NEU               4106 non-null   float64
 16  hour              4106 non-null   int64  


In [ ]:
df_timestamp.to_csv('Data/data_after_preprocessing_text_sa_timestamp.csv')